## Install Libraries


In [33]:
%pip install "ray[tune]"
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libraries


In [34]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict


## Data loaders


In [35]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [36]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [37]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 32
channel_2 = 64
channel_3 = 128
pool_kernel_size = 2

kernel_size_1 = 3
kernel_size_2 = 5
kernel_size_3 = 7

pad_size_1 = 2
pad_size_2 = 3
pad_size_3 = 3

fc_count_1 = 1024

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [38]:
def train_cifar(config):
    net = Net()  # Will need to modify the parameters depending on what you are experimenting with

    learning_rate = config['learning_rate']
    momentum = 0.5
    weight_decay = 1e-4

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    

    criterion = nn.CrossEntropyLoss()

    if config['opt'] == 'sgd':
      optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                        weight_decay=weight_decay,
                        momentum=0, nesterov=False)
    
    elif config['opt'] == 'sgd_nes':
      optimizer = optim.SGD(net.parameters(), lr=learning_rate, 
                  weight_decay=weight_decay,
                  momentum=0, nesterov=False)
      
    elif config['opt'] == 'adagrad':
      optimizer = optim.Adagrad(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay)
      
    elif config['opt'] == 'rms':
      optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay, momentum=momentum)
      
    elif config['opt'] == 'adam':
      optimizer = optim.Adam(net.parameters(), lr=learning_rate, 
                      weight_decay=weight_decay)

    #optimizer = optim.SGD(net.parameters(), lr=1e-2)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [39]:
def test_best_model(best_result):
    best_trained_model = Net(
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [40]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = {"opt": tune.grid_search(['sgd', 'sgd_nes', 'adagrad', 'rms', 'adam']), 
          "learning_rate": tune.grid_search([6e-2, 4e-2, 2e-2, 1e-2, 5e-3])}



In [41]:
# Do not modify any code below!
def main(config):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)


(train_cifar pid=112437) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_07-46-48/train_cifar_2dffa_00000_0_learning_rate=0.0600,opt=sgd_2023-04-12_07-46-48/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:13<00:00, 12860836.00it/s]


(train_cifar pid=112437) Extracting /root/ray_results/train_cifar_2023-04-12_07-46-48/train_cifar_2dffa_00000_0_learning_rate=0.0600,opt=sgd_2023-04-12_07-46-48/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_07-46-48/train_cifar_2dffa_00000_0_learning_rate=0.0600,opt=sgd_2023-04-12_07-46-48/data
(train_cifar pid=112437) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_2dffa_00000,0.7525,2023-04-12_07-48-02,True,,9df85343eca142f69857be6fefe122b4,"0_learning_rate=0.0600,opt=sgd",ec71ef1e9740,10,0.937693,172.28.0.12,112437,True,70.9051,4.91554,70.9051,1681285682,0,,10,2dffa_00000,0.00400162
train_cifar_2dffa_00001,0.7679,2023-04-12_07-49-10,True,,9df85343eca142f69857be6fefe122b4,"1_learning_rate=0.0400,opt=sgd",ec71ef1e9740,10,0.861384,172.28.0.12,112437,True,67.5219,4.95054,67.5219,1681285750,0,,10,2dffa_00001,0.00400162
train_cifar_2dffa_00002,0.7536,2023-04-12_07-50-17,True,,9df85343eca142f69857be6fefe122b4,"2_learning_rate=0.0200,opt=sgd",ec71ef1e9740,10,0.859859,172.28.0.12,112437,True,66.8174,4.82784,66.8174,1681285817,0,,10,2dffa_00002,0.00400162
train_cifar_2dffa_00003,0.735,2023-04-12_07-51-23,True,,9df85343eca142f69857be6fefe122b4,"3_learning_rate=0.0100,opt=sgd",ec71ef1e9740,10,0.830985,172.28.0.12,112437,True,66.5761,4.79484,66.5761,1681285883,0,,10,2dffa_00003,0.00400162
train_cifar_2dffa_00004,0.7241,2023-04-12_07-52-30,True,,9df85343eca142f69857be6fefe122b4,"4_learning_rate=0.0050,opt=sgd",ec71ef1e9740,10,0.815518,172.28.0.12,112437,True,66.9281,4.68343,66.9281,1681285950,0,,10,2dffa_00004,0.00400162
train_cifar_2dffa_00005,0.7646,2023-04-12_07-53-36,True,,9df85343eca142f69857be6fefe122b4,"5_learning_rate=0.0600,opt=sgd_nes",ec71ef1e9740,10,0.922352,172.28.0.12,112437,True,65.6858,4.66239,65.6858,1681286016,0,,10,2dffa_00005,0.00400162
train_cifar_2dffa_00006,0.7468,2023-04-12_07-54-42,True,,9df85343eca142f69857be6fefe122b4,"6_learning_rate=0.0400,opt=sgd_nes",ec71ef1e9740,10,0.930633,172.28.0.12,112437,True,66.1114,4.75691,66.1114,1681286082,0,,10,2dffa_00006,0.00400162
train_cifar_2dffa_00007,0.7096,2023-04-12_07-55-52,True,,9df85343eca142f69857be6fefe122b4,"7_learning_rate=0.0200,opt=sgd_nes",ec71ef1e9740,10,1.04738,172.28.0.12,112437,True,70.0672,4.77523,70.0672,1681286152,0,,10,2dffa_00007,0.00400162
train_cifar_2dffa_00008,0.7388,2023-04-12_07-56-57,True,,9df85343eca142f69857be6fefe122b4,"8_learning_rate=0.0100,opt=sgd_nes",ec71ef1e9740,10,0.813094,172.28.0.12,112437,True,65.3148,4.75657,65.3148,1681286217,0,,10,2dffa_00008,0.00400162
train_cifar_2dffa_00009,0.728,2023-04-12_07-58-05,True,,9df85343eca142f69857be6fefe122b4,"9_learning_rate=0.0050,opt=sgd_nes",ec71ef1e9740,10,0.83487,172.28.0.12,112437,True,67.5177,4.83101,67.5177,1681286285,0,,10,2dffa_00009,0.00400162


(train_cifar pid=112437) Finished Training
(train_cifar pid=112437) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_07-46-48/train_cifar_2dffa_00001_1_learning_rate=0.0400,opt=sgd_2023-04-12_07-48-02/data/cifar-10-python.tar.gz


...

(train_cifar pid=112437) Extracting /root/ray_results/train_cifar_2023-04-12_07-46-48/train_cifar_2dffa_00023_23_learning_rate=0.0100,opt=adam_2023-04-12_08-13-31/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_07-46-48/train_cifar_2dffa_00023_23_learning_rate=0.0100,opt=adam_2023-04-12_08-13-31/data
(train_cifar pid=112437) Files already downloaded and verified
(train_cifar pid=112437) Finished Training
(train_cifar pid=112437) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_07-46-48/train_cifar_2dffa_00024_24_learning_rate=0.0050,opt=adam_2023-04-12_08-14-42/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:16<00:00, 10409471.21it/s]


(train_cifar pid=112437) Extracting /root/ray_results/train_cifar_2023-04-12_07-46-48/train_cifar_2dffa_00024_24_learning_rate=0.0050,opt=adam_2023-04-12_08-14-42/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-12_07-46-48/train_cifar_2dffa_00024_24_learning_rate=0.0050,opt=adam_2023-04-12_08-14-42/data
(train_cifar pid=112437) Files already downloaded and verified


2023-04-12 08:15:54,784	INFO tune.py:798 -- Total run time: 1746.48 seconds (1746.45 seconds for the tuning loop).


(train_cifar pid=112437) Finished Training
Best trial config: {'opt': 'sgd', 'learning_rate': 0.04}
Best trial final validation loss: 0.8613837703018431
Best trial final validation accuracy: 0.7679
Files already downloaded and verified
Files already downloaded and verified
Best trial test set accuracy: 0.7377


In [42]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/opt
1,0.861384,0.7679,4.950539,False,67.521874,0.040,sgd
5,0.922352,0.7646,4.662392,False,65.685817,0.060,sgd_nes
0,0.790352,0.7626,4.917248,False,61.076888,0.060,sgd
6,0.720641,0.7590,4.804915,False,51.770998,0.040,sgd_nes
2,0.859859,0.7536,4.827842,False,66.817440,0.020,sgd
7,0.786104,0.7524,4.701315,False,65.291944,0.020,sgd_nes
3,0.753906,0.7510,4.809244,False,61.781293,0.010,sgd
14,0.842639,0.7456,5.383120,False,73.301125,0.005,adagrad
24,0.807828,0.7394,5.030855,False,72.149987,0.005,adam
8,0.813094,0.7388,4.756567,False,65.314794,0.010,sgd_nes


In [43]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/learning_rate,config/opt
16,95.911038,0.1010,5.029337,False,58.150253,0.040,rms
20,2.563361,0.1019,5.126347,False,48.344251,0.060,adam
15,190.961538,0.1040,4.946035,False,64.229026,0.060,rms
17,246.195076,0.1046,22.712910,False,22.712910,0.020,rms
18,2.329547,0.1048,4.909046,False,55.113385,0.010,rms
19,2.148173,0.1922,4.967229,False,50.002054,0.005,rms
10,2.058584,0.3446,5.631820,False,74.970793,0.060,adagrad
21,1.609014,0.4304,5.139658,False,71.336181,0.040,adam
11,1.390852,0.5015,5.671218,False,74.313518,0.040,adagrad
22,1.099801,0.6132,5.139256,False,65.543417,0.020,adam
